In [41]:
import itertools
import sys
from nltk.grammar import Nonterminal

In [44]:
def generate(grammar, start=None, depth=None, n=None):
    """
    Generates an iterator of all sentences from a CFG.

    :param grammar: The Grammar used to generate sentences.
    :param start: The Nonterminal from which to start generate sentences.
    :param depth: The maximal depth of the generated tree.
    :param n: The maximum number of sentences to return.
    :return: An iterator of lists of terminal tokens.
    """
    if not start:
        start = grammar.start()
    if depth is None:
        depth = sys.maxsize

    iter = _generate_all(grammar, [start], depth)

    if n:
        iter = itertools.islice(iter, n)

    return iter



def _generate_all(grammar, items, depth):
    if items:
        try:
            for frag1 in _generate_one(grammar, items[0], depth):
                for frag2 in _generate_all(grammar, items[1:], depth):
                    yield frag1 + frag2
        except RuntimeError as _error:
            if _error.message == "maximum recursion depth exceeded":
                # Helpful error message while still showing the recursion stack.
                raise RuntimeError(
                    "The grammar has rule(s) that yield infinite recursion!!"
                )
            else:
                raise
    else:
        yield []


def _generate_one(grammar, item, depth):
    if depth > 0:
        if isinstance(item, Nonterminal):
            for prod in grammar.productions(lhs=item):
                for frag in _generate_all(grammar, prod.rhs(), depth - 1):
                    yield frag
        else:
            yield [item]

O =   ["'name'", "'date of birth'", "'first name'", "'last name'", "'amount owed'", "'address'", "'registration number'", "'company number'"]
Os = (" [{0}] | ".format(1 / len(O))).join(O)            
Os += " [{0}] ".format(1 / len(O))
print(Os)

demo_grammar = """
  S -> QP T [1.0]
  QP -> Q PR [1.0]
  PR -> Det OP [0.333] | Det O Conj O Pos Det O2 [0.333] | Det O Comm O Conj O Pos Det O2 [0.333]
  OP -> O Pos Det O2 [0.5] | O2 Ap O [0.5]
  Q -> 'what is' [1] 
  Det -> 'the' [1.0]
  Conj -> 'and' [1.0] 
  Comm -> ',' [1.0]
  O -> """
demo_grammar += Os
demo_grammar += """
  Pos -> 'of' [0.9] | 'to' [0.1]
  Ap -> 's' [1.0]
  O2 -> [0.166666666] 'employee' | [0.166666666] 'customer' | [0.166666666] 'client' | [0.166666666] 'company' | [0.166666666] 'payee' | [0.166666666] 'recipient'
  T -> '?' [1.0]
"""
demo_grammar


'name' [0.125] | 'date of birth' [0.125] | 'first name' [0.125] | 'last name' [0.125] | 'amount owed' [0.125] | 'address' [0.125] | 'registration number' [0.125] | 'company number' [0.125] 


"\n  S -> QP T [1.0]\n  QP -> Q PR [1.0]\n  PR -> Det OP [0.333] | Det O Conj O Pos Det O2 [0.333] | Det O Comm O Conj O Pos Det O2 [0.333]\n  OP -> O Pos Det O2 [0.5] | O2 Ap O [0.5]\n  Q -> 'what is' [1] \n  Det -> 'the' [1.0]\n  Conj -> 'and' [1.0] \n  Comm -> ',' [1.0]\n  O -> 'name' [0.125] | 'date of birth' [0.125] | 'first name' [0.125] | 'last name' [0.125] | 'amount owed' [0.125] | 'address' [0.125] | 'registration number' [0.125] | 'company number' [0.125] \n  Pos -> 'of' [0.9] | 'to' [0.1]\n  Ap -> 's' [1.0]\n  O2 -> [0.166666666] 'employee' | [0.166666666] 'customer' | [0.166666666] 'client' | [0.166666666] 'company' | [0.166666666] 'payee' | [0.166666666] 'recipient'\n  T -> '?' [1.0]\n"

In [45]:
from nltk.grammar import CFG, PCFG
N = 10000
print('Generating the first %d sentences for demo grammar:' % (N,))
print(demo_grammar)
grammar = PCFG.fromstring(demo_grammar)
with open('./generated.out', 'w') as outfile:
    for n, sent in enumerate(generate(grammar, n=N), 1):
        print('%3d. %s' % (n, ' '.join(sent)))
        #outfile.write(' '.join(sent) + '\n')

Generating the first 10000 sentences for demo grammar:

  S -> QP T [1.0]
  QP -> Q PR [1.0]
  PR -> Det OP [0.333] | Det O Conj O Pos Det O2 [0.333] | Det O Comm O Conj O Pos Det O2 [0.333]
  OP -> O Pos Det O2 [0.5] | O2 Ap O [0.5]
  Q -> 'what is' [1] 
  Det -> 'the' [1.0]
  Conj -> 'and' [1.0] 
  Comm -> ',' [1.0]
  O -> 'name' [0.125] | 'date of birth' [0.125] | 'first name' [0.125] | 'last name' [0.125] | 'amount owed' [0.125] | 'address' [0.125] | 'registration number' [0.125] | 'company number' [0.125] 
  Pos -> 'of' [0.9] | 'to' [0.1]
  Ap -> 's' [1.0]
  O2 -> [0.166666666] 'employee' | [0.166666666] 'customer' | [0.166666666] 'client' | [0.166666666] 'company' | [0.166666666] 'payee' | [0.166666666] 'recipient'
  T -> '?' [1.0]

  1. what is the name of the employee ?
  2. what is the name of the customer ?
  3. what is the name of the client ?
  4. what is the name of the company ?
  5. what is the name of the payee ?
  6. what is the name of the recipient ?
  7. what is the

2134. what is the date of birth , amount owed and address to the company ?
2135. what is the date of birth , amount owed and address to the payee ?
2136. what is the date of birth , amount owed and address to the recipient ?
2137. what is the date of birth , amount owed and registration number of the employee ?
2138. what is the date of birth , amount owed and registration number of the customer ?
2139. what is the date of birth , amount owed and registration number of the client ?
2140. what is the date of birth , amount owed and registration number of the company ?
2141. what is the date of birth , amount owed and registration number of the payee ?
2142. what is the date of birth , amount owed and registration number of the recipient ?
2143. what is the date of birth , amount owed and registration number to the employee ?
2144. what is the date of birth , amount owed and registration number to the customer ?
2145. what is the date of birth , amount owed and registration number to the

4265. what is the amount owed , first name and company number of the payee ?
4266. what is the amount owed , first name and company number of the recipient ?
4267. what is the amount owed , first name and company number to the employee ?
4268. what is the amount owed , first name and company number to the customer ?
4269. what is the amount owed , first name and company number to the client ?
4270. what is the amount owed , first name and company number to the company ?
4271. what is the amount owed , first name and company number to the payee ?
4272. what is the amount owed , first name and company number to the recipient ?
4273. what is the amount owed , last name and name of the employee ?
4274. what is the amount owed , last name and name of the customer ?
4275. what is the amount owed , last name and name of the client ?
4276. what is the amount owed , last name and name of the company ?
4277. what is the amount owed , last name and name of the payee ?
4278. what is the amount owe

6642. what is the company number , last name and address of the recipient ?
6643. what is the company number , last name and address to the employee ?
6644. what is the company number , last name and address to the customer ?
6645. what is the company number , last name and address to the client ?
6646. what is the company number , last name and address to the company ?
6647. what is the company number , last name and address to the payee ?
6648. what is the company number , last name and address to the recipient ?
6649. what is the company number , last name and registration number of the employee ?
6650. what is the company number , last name and registration number of the customer ?
6651. what is the company number , last name and registration number of the client ?
6652. what is the company number , last name and registration number of the company ?
6653. what is the company number , last name and registration number of the payee ?
6654. what is the company number , last name and r

In [48]:
import re
from bs4 import BeautifulSoup
from os import listdir
from os.path import isfile, join
import random
from random import randint
import datetime


path = "/virtualmachines/data/companies"
files = [join(path,f) for f in listdir(path) if isfile(join(path, f)) and f.endswith(".xml")]

def generate_names():
    with open(random.choice(files), "r") as infile:
        for line in infile:
            z = re.search(r'<NonIndividualNameText>([^</]+)</NonIndividualNameText>',line)
            if z:
                yield z.group(1).replace("&amp;", "&")
                
def generate_date():
    formats = ['%d, %b %Y', '%d %b %Y', '%d %B %Y', '%d %m %Y','%d-%m-%Y', '%d/%m/%Y','%Y-%m-%d', '%Y/%m/%d']
    return datetime.date(randint(1950,2025), randint(1,12),randint(1,28)).strftime(random.choice(formats))

    
def generate_amount():
    ret_string = "";
    if random.random() > 0.5:
        ret_string = random.choice(["US","AU","A", "US$", "$US", "AU$", "$AU", "A$", "$A", "GBP", "GBP£", "£GBP"])
        
    if random.random() > 0.5:
        ret_string = ret_string + "{:,}".format(random.randint(0,100000))
    else:
        ret_string = ret_string + "{:}".format(random.randint(0,100000))
    
    if random.random() > 0.25:
        ret_string += "." + str(random.randint(0,99));
    return ret_string

for i in range(100):
    print(generate_date())

04/11/2024
1974-03-07
2024/01/07
16 08 2021
27-04-1953
2021/11/27
26 January 1973
07/04/2006
02-03-2014
26-10-1967
2022-07-16
10/03/1987
15-11-1966
1958/06/07
08, Jul 1976
1974/03/07
10 November 2024
15 09 1961
25/02/2002
17, Nov 2001
09/12/1981
23-02-1990
07 April 1950
1967/10/24
28/07/1995
2023-06-07
2014/04/09
05 Mar 2016
25 11 2006
2023-02-07
16 07 1996
23 Nov 2003
28/12/2010
20 July 1969
17/12/1964
13, Jul 1990
1978/03/26
1989/08/07
12-12-1971
23 December 1950
1995/05/23
21 June 1994
27 Aug 1967
1984-03-08
22 July 2012
1972/09/02
11 07 1993
06 Sep 1977
26-06-1986
21/05/1968
21/01/2011
07 08 1991
04, Jul 1980
18/06/1977
09 December 2019
18-01-1998
2003/09/11
1969-03-05
27, Nov 2015
18 Apr 2020
1991/08/15
06/01/1955
1965/05/23
20 May 1991
06 June 1987
26-09-2021
25/07/1955
14 10 2012
05/08/1976
12-12-2011
2023/05/03
27 May 1979
10-12-2025
2002/11/16
20 April 1963
20 January 1972
1991-02-09
2008/08/11
18 Sep 1950
27-11-1991
08 May 1984
16-05-1993
25 05 2002
21 06 1956
1981-06-19
03 O

In [21]:
#with open(random.choice(files), "r") as infile:
#    soup = BeautifulSoup(infile, 'xml')



PMB FEEDLOT PTY LIMITED
KHEM'S CLEANING PTY LTD
LA BARAKA PTY LTD
QUARTZ ISLAND PTY LTD
KHALEEJ @ ALMINA PTY LTD
Cathy Hewett
SALUTE SUPERANNUATION PTY LTD
RHYTHM EVENTS PTY LTD
JHF INVESTMENTS PTY LTD
POSSIBILITY PLUS PTY. LTD.
GEEWHY
The trustee for BarlettaSuper
SU MYAT PTY LTD
GOLD MAX AUS PTY. LTD.
BUSINESS FRUITION PTY LTD
KOTLAR DENTAL PTY LTD
TONY CINQUE &amp; FERNANDO E DE SANTIS
EMILY'S CLEANING SERVICES
ACTIVE APPS PTY LTD
Australian Association of Psychologists Inc
JOHN'S SUPER FUND PTY LTD
The Trustee for Reservoir Smiles Unit Trust
M &amp; J &amp; L TRADING PTY. LTD.
JACARANDAH INVESTMENTS PTY LTD
SATINDER SINGH PTY LTD
The Trustee for JAMES DONCAS FAMILY TRUST
Rada Studios
GUFF GUFF
NAVIR PTY LTD
MARK TRENDS PTY LTD
CANDY APPLE HOLDINGS PTY. LTD.
A &amp; A GRECH PTY. LTD.
MY FASHION REPUBLIC PTY LTD
FG CORPORATE FINANCE PTY LTD
NUSOURCE PTY LTD
ADMINISTRATIVE SERVICES INFINITY PTY LIMITED
AUSECON PTY. LTD.
The trustee for TE &amp; KP Prisk Family Trust
LEDACON GROUP PTY 

VINCENT CALVI
La Bella Sin
BOYUMU PTY LTD SUPERANNUATION FUND
The Trustee for the James Pollock Trust
VIDONI FAMILY TRUST
C &amp; T TRADING
The Trustee for Mirza Family Trust
E CHANG &amp; C HSU &amp; K LIANG &amp; A PATWIN
C.E ENRIGHT &amp; M.E ENRIGHT
THE TRUSTEE FOR THE TAN FAMILY TRUST
K.M LAWSON &amp; M.P LAWSON
Jim's Mowing &amp; Garden Care (Austins Ferry)
C.M REDMOND &amp; J.M REDMOND
Acorn Ridge
PATRICK J &amp; BRONWYN A MURPHY
ALT Bookkeeping
Coconut Grove
AQUA BLUE CARE SERVICES
revie industries
Jenny Ayres
Youi Excavations
DAVID L &amp; CORAL A MCCALMAN
STEVEN J &amp; KAREN L ROW
The Trustee for Haziraj Family Trust
The Trustee for TREE CARE MACHINERY TRUST
The Trustee for THE NANKING TRUST
RJ &amp; K SMITH
The trustee for Sommee Family Trust
BRIAN ETHERTON
Gild and Grace
The Pidgin Initiative
MEANNICE PTY LTD &amp; P TY
The Trustee for W &amp; D Blay Superannuation Fund
K BLANCHARD &amp; D.R FRASER
Ada ku
MR DWAYNE COCHRANE
THE TRUSTEE FOR CAISSA TRUST
The Mystery Ice Crea

WARWICK E LATTER
STEVE KAVANAGH
ACACIA DOWNS
Central Coast Computers &amp; Furniture
SUNSHINE KIDS &amp; INFANTS PHYSIOTHERAPY
YEN INDUSTRIES
THE TRUSTEE FOR RODGER GRIFFITHS SUPERANNUATION FUND
GALART NO1 TRUST
Danica Lee
The Trustee for LIVINGSTON FAMILY TRUST
BLUE GATE TECHNOLOGY
ROBERT J SYNON &amp; MICHAEL D SCOTT
The Trustee for The McKay Property Trust
The trustee for the Warden Family Trust
TAHNEE PHOTOGRAPHY
D.N LOCKETT &amp; G.R LOCKETT
Professional Certifiers
MITCHELHILL FAMILY TRUST
D.G ALEXANDER &amp; T ALEXANDER
John Hurley
The C Kunspyruhzy LLC
THE BAREFOOT WAY
John Charles Saligari
Daniel Desmond
The Trustee for the D R Dix Farm Trust
The Trustee for Kymitch Superfund
Screentactic
SULIN ACUPUNCTURE &amp; CHINESE MEDICINE CENTRE
P D &amp; D J WRIGHT
G.A MCKAY &amp; R.E MCKAY
P D &amp; C V O'CONNOR
PETER LEPP
TWW CAR DETAILING
T Palmer Bookkeeping
The Trustee for The Victor Knight Super Fund
STUART MORA
ALPHA HANDYMAN SERVICES
THE WHYBURN FAMILY SETTLEMENT
B PHILLIPS &amp

The trustee for PILLAI SUPERFUND
CABOOLTURE CHRISTIAN SPIRITUAL CHURCH INCORPORATED
The Trustee for Sathish &amp; Radhika Family Trust
NICHOLSON SUPERANNUATION FUND
T A RICHARDS
The trustee for Clark Horizon Superannuation Fund
WILLIAM MCCABE
Kaarl Aboriginal Corporation of Resource Development
Bathurst Community Climate Action Network Incorporated
D AMOS &amp; K.G AMOS &amp; I.R TRELOAR &amp; M.I TRELOAR
Top Hand Car Wash Detailing &amp; Service
Jim's Mowing South Morang North
G.R WATSON &amp; T WATSON
PETER GEORGE TATHAM
Sullivan Environment Consulting
Caring for Wollondilly
The Trustee for Hardman Family Super Fund
Adventure Bound 4WD Repairs
The Trustee for PCM UNIT TRUST
KARLIE PRICE DESIGN
TJ &amp; MG L'BARROW
The Trustee for Kerr Land Trust No 2
The Trustee for Zocar Lucky Family Trust
The Trustee for HOFFMAN FAMILY TRUST
A.S PEDERICK &amp; R.A PEDERICK
MONKEY SOFTWARE SUPERANNUATION FUND
NSM Contracting
HICKLING RONALD C &amp; ROBIN S
Donald Miller
TIFFANY L RICHARDS &amp; LOUI

ARTHUR WATSON ESTATE
VASILIOS BORAS &amp; HO PONG YUEN &amp; SUI PING GAO
R &amp; A SULLY
CANDY'S HAIR &amp; BEAUTY SALOON
Costume Cottage
C.M PRINCE &amp; R.J PRINCE
The Trustee for Grumball Family Trust
The Trustee for G Wiz Superannuation Fund
Love is in the Hair
Conrad Construction
The Trustee for Morrison Family Super Fund
STAR PARTS
GAUNSON-O'BRIEN SUPERANNUATION FUND
The trustee for The Olchowik Family Trust
FixRite
Yarra Valley Plumbing
A.H KIRKUP &amp; J.L KIRKUP
PA &amp; MK JONES
The Trustee for SK &amp; YT LEONG FAMILY TRUST
S.V PETER PAUL &amp; E.S SURIYA ARACHCHIGE
Jannies Cleaning Services (Steve Neville)
Dara Clear
JASON S MCDONALD &amp; BRADLEY J WINSLADE
Huxley Hodgson
The trustee for Jacob Family Trust
SUNSHINE HEALING
JOHN BURLES AIRCONDITIONING
A.J RAMSAY &amp; A.C WILLIS
TANNER P F &amp; L F
The Trustee for The Guiri Family Trust
Dot &amp; Lily Designs
RW &amp; HA SHAND SUPERANNUATION FUND
WOZZA'S ROOFING
The Trustee for Jusca Family Trust
FRESHWATER STATE SCHOOL P

...

STAN &amp; IRENE TOTSIS
The Trustee for THE TOULLARD UNIT TRUST
The Trustee for Jason C Ball
TRANFAC SUPERANNUATION FUND
MCDONALD MURPHY PROPERTY TRUST
Stephen Stransman
vaughan william south
Richard Makhlouf
GEOGRAPHE PIZZA &amp; TAKEAWAY
Christopher Midson
Bell Bird Boutique
The Trustee for THE TUKURUA TRUST
The trustee for The Thule Unit Trust
THE RAFT FAMILY TRUST
Glenda 20
E Kovac Services
Strata Plan 97194
The Trustee for SKIF Superannuation Fund
JASMINE WELLS DESIGN
Civic Seating PROPRIETARY &amp; The Trustee for The Ken Baker Family Trust
The Trustee for Mitchell Family Superannuation Fund
C AND M PAPOULIS FAMILY TRUST
The Trustee for M &amp; J Turner Family Trust
MONIQUE  CARUSI
J &amp; B KARIA FAMILY TRUST
IAN LACKENBY FAMILY TRUST
AUSTCINE PUBLISHING
G &amp; M AMES
G R PARKER
McCORMACK RED HILL SUPER FUND
KYNC
The Trustee for DF &amp; VD CAHILL SUPERANNUATION FUND
RG &amp; MR GILGEN
PARAMJIT KAUR
GroutPro Robina
Sunshine Kids Family Daycare
J JAURA &amp; S JAURA
The trustee 

HONEYHEART HOLDINGS PTY LTD
James Computers
The Trustee for CBC Staff Selection (Townsville)
Kode-Ninjas
The Trustee for THE BRADLEY MAGENNIS FAMILY TRUST
GLO-X PTY LTD
SIMPLE SCAFFOLD SOLUTIONS PTY LTD
The Trustee for C&amp;J Yates Superannuation Fund
WOORI MAINTENANCE PTY LTD
A6W8 PTY. LTD.
HICKORY BUILD PTY. LTD.
AROMAS LEASING MURARRIE PTY LTD
DESWIK GEOLOGY SOFTWARE PTY LTD
The Assistant WA
ANNABEL STANTON
WILD OATS YACHTING SERVICES PTY LTD
3MS (QLD) PTY LTD
SUGDEN &amp; ASSOCIATES PTY LTD
HC PROPERTIES PTY LTD
Concrete werx Australia
The Trustee for THE KAY FAMILY TRUST
BRENDAN LEWIS
Singh IT Solutions
CORPORATE RECOVERY SERVICES PTY LTD
MCKERNAN RESTORATION PTY LTD
ARCHER ECOLOGY PTY. LTD.
WXL01 PTY LTD
RA STAR ENTERPRISES PTY LTD
FAFAGEL HOLDINGS PTY LTD
METROPOLIS EQUITY PTY. LTD.
RIVI CAPITAL GROUP PTY LTD
FERLY JUNGLE PTY. LTD.
CLOUD TRADING PTY LTD
KARTER'S HAULAGE PTY LTD
BELARNA PTY LTD SUPER PENSION FUND
C. D. BARNETT
BUSINESS ADMINISTRATION AND LOGISTIC SERVICES PTY. L

KESBY INVESTIGATIONS GROUP PTY LTD
PIRIMID CONSTRUCTIONS PTY LTD
AJAX CONSULTING PTY LTD
OZZY ROOFING PTY LTD
Yamba Building Design
MAVERICK PARTNERS PTY LTD
ZINTU PTY LTD
TRAVIS GROUP PTY LTD
BLUE VOODOO DESIGNS
HOSEN PTY LTD
L C YEH PTY. LTD.
YANAKI PTY LIMITED
joseph clegg
AMALAIN PTY LTD
DEVIREDDY'S PTY. LTD.
COAST TO COUNTRY CARPENTRY AN CONTRACTING PTY LTD
PETERSEN SUPERANNUATION FUND
danveer singh
MARILYN'S SUPER FUND PTY LTD
HABITAT NTH PTY LTD
PAD EQUITY PTY. LTD.
SINA CONSULTANCY PTY LTD
The Trustee for LITHRANGOMITES FAMILY TRUST
TANG'S GROUP (AUSTINA) PTY LTD
TATE17 FINANCIAL SERVICES PTY LTD
ELYSTAN CONSULTING PTY LTD
THE A A ARMSTRONG FAMILY TRUST
G S KHEHRA TRANSPORT PTY. LTD.
L-EMOTION PTY. LTD.
Robert Unasa
STENNER HOLDINGS PTY LTD
MIDTOWN SERVICES AUSTRALIA PTY LTD
BLUE OCEANZ PTY LTD
SK TOLL SUPER PTY LTD
N V TRAN
KIMBERLY RESTAURANT PTY LTD
TELECORP PTY LTD
SHE DESIGNS
Adro Clean
SAM NHI AND SONS PTY LTD
COLLECTIVE SOL PTY LTD
CRAFT NATION PTY LTD
Jason Hughes
SHARP

THE OIL MAN PTY LIMITED
VICTOR &amp; ALMA'S PTY. LTD.
Mz plastering
MANUSHYA PTY LTD
GIBSON TRUCK &amp; TRANSPORT PTY LTD
BURLEIGH HEADS CANNABIS PTY LTD
LEAP &amp; CO PTY. LTD.
LEBLON CLINIC PTY LTD
BIOGENETICS AUSTRALIA PTY LTD
SKYLAND ONE PTY LTD
MOBI HEALTHCARE PTY LTD
INTRINSIC DIAMOND INVESTMENT CENTRE PTY LTD
MELBOURNE BAR TOURS TRADING PTY LTD
A.C.N. 615 921 296 PTY LTD
KINSHIP CAMPS PTY LTD
DE-CONSTRUCT AND RECYCLE PTY LTD
NPK AGRICULTURE PTY LTD
SYNVEST PROPERTY PTY LTD
CLARE FINANCIAL SERVICES PTY LTD
SCORPION PEST MANAGEMENT PTY LTD
AUA INTERNATIONAL PTY LTD
FINHASA PTY LTD
VENDBERG GROUP PTY. LTD.
TECHLINK DESIGN PTY LTD
1308 ALBANY HIGHWAY PTY LTD
CAMP &amp; CATERING PTY LTD
MALI LOSINJ PTY LTD
GREGORY ALAN &amp; SUZANNE JANE SMITH
CASTLEREAGH WATCH AND SECURITY PTY LIMITED
GEN2 AUSTRALIA PTY LTD
ASIF AAYAN PTY. LTD.
MACEDON MARBLE PTY LTD
The trustee for Kevin &amp; Anne Taylor Superannuation Fund
TOTAL SAFE PTY LTD
JMC Cleaning
THE TRUSTEE FOR POSSUM MAGIC SUPER
COSME V

KeyboardInterrupt: 

In [ ]:
  Q -> 'what' [0.1428] | 'who' [0.1428] | 'where' [0.1428] | 'when' [0.1428] | 'how much' [0.1428] | 'will' [0.1428] | 'which' [0.1428]
